In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fungiclef.evaluate import predict, evaluate
from fungiclef.model.utils import get_timm_model
from torch.utils.data import DataLoader, Dataset

In [3]:
import torch
import timm
import torch.nn as nn
import cv2
import numpy as np
import pandas as pd

In [4]:
# Variables to file path - to edit / change as necessary
VAL_PARQUET = "../dev_val.parquet"
DF20_PRETRAINED = "../DF20-ViT_large_patch16_224_best_accuracy.pth"

In [5]:
## Extract ground truth class IDs
val_df = pd.read_parquet(VAL_PARQUET)
val_df[['observationID', 'class_id']].drop_duplicates("observationID").to_csv("../tmp/gt_id.csv", index=False)

Example - Using pretrained weights from DF20 team

In [19]:
MODEL_NAME = 'vit_large_patch16_224'
model = get_timm_model(MODEL_NAME, 1604, pretrained=False)

weights = torch.load(DF20_PRETRAINED)

In [20]:
n_weights = {k.replace("module.", ""): v for k, v in weights.items()}
model.load_state_dict(n_weights)

<All keys matched successfully>

In [23]:
PRED_OUTPUT = "../tmp/pred_id.csv"
model.eval()
predict.run_inference_vit(val_df, PRED_OUTPUT, model)

Creating DataLoader.
Running inference.
Using device: cuda


100%|██████████| 30/30 [00:03<00:00,  8.16it/s]

Saving predictions to '../tmp/pred_id.csv'.


In [24]:
# This is a direct port of the official evaluate function the team will be using
evaluate.evaluate_csv(test_annotation_file="../tmp/gt_id.csv", user_submission_file="../tmp/pred_id.csv")

Evaluated scores: {'F1 Score': 1.66, 'Track 1: Classification Error': 0.6285, 'Track 2: Cost for Poisonousness Confusion': 11.8283, 'Track 3: User-Focused Loss': 12.4568, 'Track 4: Classification Error with Special Cost for Unknown': 0.6285}


[{'test_split': {'F1 Score': 1.66,
   'Track 1: Classification Error': 0.6285,
   'Track 2: Cost for Poisonousness Confusion': 11.8283,
   'Track 3: User-Focused Loss': 12.4568,
   'Track 4: Classification Error with Special Cost for Unknown': 0.6285}}]